# How to analyze text using Azure Content Safety API

Import the necessary libraries for Content Safety

In [12]:
import os
import json
import openai
from dotenv import load_dotenv
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions
from azure.ai.contentsafety.models import TextCategory

Set Azure OpenAPI connection variables

In [13]:
openai.api_type = "azure"
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
openai.api_key = os.environ["AZURE_OPENAI_KEY"]
openai.api_version = "2023-10-01-preview"

Enter the user's input text

In [14]:
user_input = "What is John's job in the movie John Wick?"

Contruct the prompt defining what the chatbot does.  If there are any rules on what the chat should or should not do, add it in the system content.

In [15]:

print(os.environ["GPT_35_TURBO_DEPLOYMENT"])
print(os.environ["AZURE_CONTENTSAFETY_ENDPOINT"])

gpt35TurboDply
https://az-contentsafety-wk.cognitiveservices.azure.com/


In [16]:
response = openai.ChatCompletion.create(
    engine=os.environ["GPT_35_TURBO_DEPLOYMENT"], # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f'{user_input}'}
    ]
)

Get OpenAPI response for Azure OpenAPI

In [17]:
openai_answer = response['choices'][0]['message']['content']

Set key and endpoint for your content safety API

In [18]:
endpoint_url=os.environ["AZURE_CONTENTSAFETY_ENDPOINT"]
credential_key=AzureKeyCredential(os.environ["AZURE_CONTENTSAFETY_KEY"])

Create a client for Content Safety and authenticate with Azure Key Credential

In [19]:
client = ContentSafetyClient(
    endpoint=endpoint_url,
    credential=credential_key
)

Submit openAI response to analyze for offensive content

In [20]:
request = AnalyzeTextOptions(text=openai_answer)

Call the content safety API to analyze a text

In [21]:
# Analyze text
try:
    response = client.analyze_text(request)
except HttpResponseError as e:
    print("Analyze text failed.")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

Loop through the API response to retrieve the Content Safety categories.

hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

Print the content safety analysis results.  View which categories were flagged and they severity level.

In [22]:
if hate_result:
    print(f"Hate severity: {hate_result.severity}")
if self_harm_result:
    print(f"Self-Harm severity: {self_harm_result.severity}")
if sexual_result:
    print(f"Sexual severity: {sexual_result.severity}")
if violence_result:
    print(f"Violence severity: {violence_result.severity}")

Hate severity: 0
Self-Harm severity: 0
Sexual severity: 0
Violence severity: 2


: 

Great Job!...You can add logic on what action to take based on the results.  For example, if the text is flagged as adult content at a severity threshold level, you can choose to not display the text.